In [6]:
from parser_1 import read_recipe_from_url
import re
import sys
import os
import spacy

nlp = spacy.load("en_core_web_lg")

folder_path = os.path.join(os.getcwd(), 'ourtypes')
sys.path.append(folder_path)

from step import Step, Steps
from ingredient import Ingredient

In [7]:
soup = read_recipe_from_url("https://www.allrecipes.com/recipe/255365/edible-cookie-dough/")

In [8]:
ingredients = soup.find("div", {"id":"mntl-structured-ingredients_1-0"})
for child in ingredients.find_all("li"):
    text = child.text
    text = text.replace("\n", "").strip()
    print(text)
    

1 cup all-purpose flour
¾ cup packed brown sugar
½ cup butter
1 teaspoon vanilla extract
½ teaspoon salt
2 tablespoons milk
½ cup milk chocolate chips
½ cup mini chocolate chips


In [12]:
ingredient_objects = {}
for child in ingredients.find_all("li"):
    text = " ".join(child.text.replace("\n", "").split()[2:])
    ingredient_objects[text] = text

steps = soup.find("div", {"id":"recipe__steps-content_1-0"})
steps_collection = Steps()

for child in steps.find_all("p"):
    step_text = child.text.replace("\n", "").strip()
    
    #find the time mentioned for in the step
    time_match = re.search(r'for (.*? (minutes?|seconds?|hours?)(\s+and\s+\d+\s*(minutes?|seconds?|hours?))?(\b|\s))', step_text)
    if time_match:
        time = time_match.group(1)
    else:
         time = None
    
    #find the temperature for that step
    temp_match = re.search(r'(\d+ degrees)', step_text)
    if temp_match:
        temp = temp_match.group(1) 
    else:
        temp = None
    
    doc = nlp(step_text)
    tools = []
    methods = []

    for sent in doc.sents:
        for token in sent:
            #check if token is a verb
            if token.pos_ == 'VERB':
                #check if it's the first token in a sentence 
                if token.i == sent.start:
                    methods.append(token.lemma_)
            
                #check if itcomes after punctuation or 'and'
                else:
                    prev_token = token.nbor(-1)
                    if prev_token.is_punct or (prev_token.text.lower() == 'and'):
                        methods.append(token.lemma_)

    #extract compound nouns for full tool names
    for chunk in doc.noun_chunks:
        tokens = chunk.text.split()
        if len(tokens) > 1 and tokens[0].lower() in ['a', 'an']:
            #check if the last word is a noun
            if nlp(chunk.text)[-1].pos_ == 'NOUN':
                tools.append(chunk.text)
        elif chunk.root.dep_ in ['dobj', 'pobj']:  #direct object or object of preposition
            #check if the current chunk is a compound noun
            if any(child.dep_ == 'compound' for child in chunk.root.children):
                tools.append(chunk.text)
            elif chunk.root.pos_ == 'NOUN':
                tools.append(chunk.text)
    #identify ingredients present in this step
    step_ingredients = []
    for ingred in ingredient_objects:
        words = ingred.split()
        for word in words:
            if word in step_text:
                step_ingredients.append(ingred)
                break

    #remove tools are ingredients, do not end in a noun and do not start with "a" or "an":
    tools = [tool for tool in tools if all(ingr not in tool for ingr in ingredient_objects)]
    tools = [tool for tool in tools if nlp(tool)[-1].pos_ == 'NOUN' and tool.split()[0] in ["a", "an"]]
    
    #create Step object
    step_obj = Step(step_ingredients, tools=tools, methods=methods, description=step_text, time=time, temp=temp)

    #add Step object to Steps collection
    steps_collection.add_step(step_obj)

print(steps_collection)

Step 1: Step(Ingredients: ['all-purpose flour'], Tools: ['a microwave-safe dish'], Methods: ['treat', 'cook', 'stir', 'set'], Time: 1 minute and 15 seconds, Temp: None, Description: 'To heat-treat your flour so it is safe to use: Place flour in a microwave-safe dish and cook for 1 minute and 15 seconds, stirring it every 15 seconds. Set aside.')
Step 2: Step(Ingredients: ['all-purpose flour', 'packed brown sugar', 'butter', 'vanilla extract', 'salt', 'milk', 'milk chocolate chips', 'mini chocolate chips'], Tools: ['an electric mixer', 'a large bowl', 'a crumbly dough forms'], Methods: ['Beat', 'Beat', 'add', 'treat', 'mix', 'stir', 'fold'], Time: None, Temp: None, Description: 'Beat sugar and butter with an electric mixer in a large bowl until creamy. Beat in vanilla extract and salt. Add heat-treated flour; mix until a crumbly dough forms. Stir in milk until dough is just combined; fold in milk chocolate chips and mini chocolate chips.')
Step 3: Step(Ingredients: ['all-purpose flour']